In [ ]:
''' import necessary libraries'''
from transformers import EsmModel, EsmTokenizer

from utils import create_dataset, get_evaluate_results, EarlyStoppingCallBack, compute_metrics, read_fasta_to_dataframe
import torch
from torch.utils.data import DataLoader
import numpy as np
from transformers import DataCollatorForTokenClassification, TrainingArguments, Trainer,set_seed
from tqdm import tqdm
import torch.nn as nn
from transformers.modeling_outputs import TokenClassifierOutput
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

/home/xie/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from models import DisoGraph
from layers import modify_with_lora, LoRAConfig
# Load pre-trained ESM-2 model and tokenizer
tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")
model = EsmModel.from_pretrained("facebook/esm2_t33_650M_UR50D")

class_model = DisoGraph(config=model.config)
class_model.encoder = model
model = class_model
del class_model

# Freeze ESM model parameters
for (param_name, param) in model.encoder.named_parameters():
    param.requires_grad = False

# Add LoRA to the model
config = LoRAConfig()
model = modify_with_lora(model, config)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print("Trainable Parameter: "+ str(params) + "\n")

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable Parameter: 1690114



/tmp/ipykernel_4148083/4264039977.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/xie/GraphIDP/model/GCN_LoRA_paramas.pth"))


<All keys matched successfully>

In [ ]:
# Preprocess inputs
train_df = read_fasta_to_dataframe("/home/xie/CEFF-IDP/Datasets/DM3000.fasta")
valid_df = read_fasta_to_dataframe("/home/xie/CEFF-IDP/Datasets/Validset.fasta")

# Create Datasets
train_set=create_dataset(tokenizer,list(train_df['sequence']),list(train_df['label']))
valid_set=create_dataset(tokenizer,list(valid_df['sequence']),list(valid_df['label']))

print(len(train_df),len(valid_df))

2954 1229


In [ ]:
import random
''' Training Definition '''
# # Set random seeds for reproducibility of your trainings run
def set_seeds(s):
    torch.manual_seed(s)
    np.random.seed(s)
    random.seed(s)
    set_seed(s)

# Huggingface Trainer arguments
args = TrainingArguments(
        "./",
        evaluation_strategy = "epoch",
        logging_strategy = "epoch",
        save_strategy = "no",
        learning_rate=3e-4,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=1,
        num_train_epochs=5,
        deepspeed= None,
        fp16 = False,
        seed= 42,
    ) 

set_seeds(42)
# For token classification we need a data collator here to pad correctly
data_collator = DataCollatorForTokenClassification(tokenizer)

# Trainer
trainer = Trainer(model, args, train_dataset=train_set, eval_dataset=valid_set, tokenizer=tokenizer,
                  data_collator=data_collator, compute_metrics=compute_metrics, callbacks=[EarlyStoppingCallBack(patience=2)])

# Train model
trainer.train()

/home/xie/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_711455/339790029.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=train_set, eval_dataset=valid_set, tokenizer=tokenizer,
/home/xie/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Acc,F1,Auc
1,0.206600,0.189760,0.939984,0.625246,0.913754
2,0.167700,0.184414,0.940646,0.633104,0.919658
3,0.159400,0.184457,0.941407,0.625607,0.921355
4,0.148700,0.185339,0.941223,0.628557,0.921698


/home/xie/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/xie/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/xie/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=988, training_loss=0.17063367511579383, metrics={'train_runtime': 3683.3462, 'train_samples_per_second': 4.01, 'train_steps_per_second': 0.335, 'total_flos': 0.0, 'train_loss': 0.17063367511579383, 'epoch': 4.0})